# ZSim Buff 系统重构项目结构与功能规划

本文档基于 `docs/Buff重构方案.md` 制定，旨在指导 ZSim 项目中 Buff 系统的解耦与重构工作。

## 1. 项目结构目录

目录结构中标注了 **[旧系统]**（将被移除/废弃）、**[新系统-现有]**（需适配）和 **[新系统-待构建]**（本次重构核心）。

```text
zsim/
├── sim_progress/
│   ├── Buff/                                   <-- [模块核心] Buff 数据与控制
│   │   ├── Buff0Manager/                       [旧系统] (将被 GlobalBuffController 取代)
│   │   │   └── Buff0ManagerClass.py            
│   │   ├── BuffXLogic/                         [旧系统] (逻辑将被拆分至 Handler 和 TriggerEffect)
│   │   │   └── ... (大量特定逻辑文件)
│   │   ├── JudgeTools/                         [旧系统] (将被 EventRouter 的逻辑树判定取代)
│   │   ├── effects/                            [新系统-待构建] (Buff 效果对象化)
│   │   │   ├── __init__.py
│   │   │   ├── base_effect.py                  (EffectBaseClass: 效果基类)
│   │   │   ├── bonus_effect.py                 (BonusEffect: 数值增减益效果)
│   │   │   └── trigger_effect.py               (TriggerEffect: 触发器/机制类效果)
│   │   ├── buff_model.py                       [新系统-待构建] (新 Buff 实体类，仅含状态)
│   │   ├── global_buff_controller.py           [新系统-待构建] (全局 Buff 仓库与工厂)
│   │   ├── buff_manager.py                     [新系统-待构建] (角色身上的 Buff 管理器)
│   │   ├── buff_class.py                       [旧系统] (将被 buff_model.py 取代)
│   │   ├── BuffLoad.py                         [旧系统] (将被初始化工厂取代)
│   │   └── BuffAdd.py                          [旧系统] (将被 BuffManager 取代)
│   │
│   ├── zsim_event_system/                      [模块核心] 事件路由与分发
│   │   ├── Handler/
│   │   │   ├── buff_handler.py                 [新系统-待构建] (Buff 专用事件处理器)
│   │   │   ├── skill_event_handler.py          [新系统-现有] (需适配)
│   │   │   └── base_handler_class.py           [新系统-现有]
│   │   ├── zsim_events/
│   │   │   ├── zsim_event.py                   [新系统-待构建] (统一事件封装)
│   │   │   ├── event_profile.py                [新系统-待构建] (事件画像，提供查询接口)
│   │   │   └── base_zsim_event.py              [新系统-现有]
│   │   ├── tree/                               [新系统-现有] (逻辑树组件)
│   │   │   ├── state_tree.py
│   │   │   └── nodes.py
│   │   └── event_router.py                     [新系统-待构建] (事件中枢)
│   │
│   └── Character/
│       ├── component/                          [新系统-待构建] (角色组件解耦)
│       │   ├── dynamic_attribute.py            (属性计算器，替代 MultiplierData)
│       │   └── bonus_pool.py                   (效果池，管理生效的 BonusEffect)
│       └── character.py                        [需修改] (接入新组件，移除旧 Buff 引用)
│
└── ...

## 2. 待构建文件详情与功能标注

以下文件需遵循 **Python SOLID 原则** 进行构建，严禁使用 `from module import *`，应明确导入依赖。

### A. Buff 核心模型与控制 (`zsim/sim_progress/Buff/`)

#### `buff_model.py`
* **类名**: `Buff`
* **功能**: Buff 实体的数据容器与状态机。
* **核心职责**:
    * **单一职责**: 仅维护 Buff 自身的生命周期状态（层数、持续时间、ID、来源），不包含任何触发判定逻辑。
    * **方法**: 
        * `start(timestamp)`: 激活/刷新 Buff。
        * `end(timestamp)`: 结束 Buff。
        * `refresh(layers, duration)`: 更新层数和时间。
    * **属性**: `ft` (静态配置), `dy` (动态数据), `effects` (持有的 Effect 对象列表)。

#### `global_buff_controller.py`
* **类名**: `GlobalBuffController`
* **功能**: 全局 Buff 管理与初始化工厂。
* **核心职责**:
    * **仓库管理**: `_buff_box` 字典存储本次模拟中所有的 Buff 实例（ID索引）。
    * **初始化工厂**: `buff_initiate_factory(sim_config)`。读取配置，实例化所有涉及的 Buff，并将其注册到 `EventRouter` 中。
    * **依赖倒置**: 不依赖具体 Simulator，仅依赖 Config 数据接口。

#### `buff_manager.py`
* **类名**: `BuffManager`, `BuffOperator`
* **功能**: 角色层面的 Buff 操作接口。
* **核心职责**:
    * **CRUD 操作**: 提供 `add_buff(buff_id)`, `remove_buff(buff_id)`, `get_active_buffs()` 接口。
    * **同步机制**: 当 Buff 列表变动时，自动通知 `BonusPool` 更新生效的效果列表。
    * **所有权**: 每个 Character 实例持有一个 `BuffManager`。

### B. Buff 效果体系 (`zsim/sim_progress/Buff/effects/`)

#### `base_effect.py`
* **类名**: `EffectBase` (Abstract Base Class)
* **功能**: 定义所有 Buff 效果的通用接口。
* **核心职责**:
    * 定义抽象方法 `apply(target)` 或属性 `value`，规范子类行为。

#### `bonus_effect.py`
* **类名**: `BonusEffect` (Inherits `EffectBase`)
* **功能**: 数值型增益/减益效果。
* **核心职责**:
    * **数据承载**: 存储 `target_attribute` (如"攻击力"), `value` (数值), `correction_type` (乘区类型)。
    * **条件约束**: 存储生效条件（如 "仅对火属性生效"）。

#### `trigger_effect.py`
* **类名**: `TriggerEffect` (Inherits `EffectBase`)
* **功能**: 机制型/触发型效果。
* **核心职责**:
    * **逻辑封装**: 处理非数值类的效果，如"造成额外伤害"、"回复能量"、"施加异常积蓄"。
    * 通常包含一个 EventID 或回调逻辑，注册到事件系统中。

### C. 事件路由系统 (`zsim/sim_progress/zsim_event_system/`)

#### `event_router.py`
* **类名**: `EventRouter`
* **功能**: 事件分发系统的核心中枢。
* **核心职责**:
    * **分发 (Routing)**: 接收 `publish(event)` 请求。
    * **解析**: 将原始事件（如 SkillNode）包装为 `EventProfile`。
    * **匹配**: 遍历 `event_trigger_tree` (逻辑树) 查找满足条件的监听器。
    * **执行**: 调用匹配到的 Handler (`HandlerMap`)。

#### `zsim_events/zsim_event.py`
* **类名**: `ZSimEvent`
* **功能**: 统一事件包装器。
* **核心职责**:
    * **适配器模式**: 将模拟器中的异构对象（`SkillNode`, `AnomalyBar`, `EnemyAction`）统一封装为标准事件对象，屏蔽底层差异。

#### `zsim_events/event_profile.py`
* **类名**: `EventProfile`
* **功能**: 事件画像（Facade）。
* **核心职责**:
    * 提供统一的、只读的查询接口供逻辑树节点使用（如 `get_skill_tags()`, `get_element_type()`）。
    * 解耦逻辑树节点与具体的事件对象。

#### `Handler/buff_handler.py`
* **类名**: `BuffEventHandler`
* **功能**: 处理 Buff 相关的具体业务逻辑。
* **核心职责**:
    * 响应 `EventRouter` 的调用，执行 Buff 的 `start/refresh` 操作。
    * 发布 `BuffUpdateEvent` 以通知 `BuffManager` 进行状态同步。

### D. 角色属性组件 (`zsim/sim_progress/Character/component/`)

#### `dynamic_attribute.py`
* **类名**: `DynamicAttribute`, `AttributeCalculator`
* **功能**: 动态属性管理与计算。
* **核心职责**:
    * **替代 MultiplierData**: 不再临时创建大对象，而是维护属性状态。
    * **计算**: `calculate(base_stat, bonus_pool)`。结合基础面板和当前效果池，计算实时属性值。

#### `bonus_pool.py`
* **类名**: `BonusPool`
* **功能**: 角色身上的增益效果池。
* **核心职责**:
    * **索引优化**: 使用二维字典或类似结构（按属性类型、来源分类）存储当前激活的 `BonusEffect`。
    
    * **查询**: 提供高效接口供 `AttributeCalculator` 快速获取指定属性的所有加成值。

# ZSim Buff 系统重构：冲突分析与施工指导指南

本文档整理了新旧 Buff 系统切换过程中可能出现的关键冲突点，以及针对复杂事件触发逻辑（原 `BuffXLogic`）的具体技术解决方案，旨在为后续代码构建提供方向指导。

---

## 第一部分：新旧系统切换的关键冲突点分析

根据 `docs/Buff重构方案.md` 与现有代码结构，以下 6 个领域是重构时的“高危区”，必须优先处理。

### 1. 核心计算逻辑冲突 (Calculator API)
**影响等级**: 🔴 **Critical** (直接导致伤害计算失效)
* **冲突描述**:
    * **旧逻辑**: `Calculator.py` 依赖旧的 `MultiplierData` 对象和 `Buff0Manager` 计算出的静态属性面板。
    * **新逻辑**: 属性计算下放至 `Character` 组件。Calculator 应通过 `AttributeCalculator.calculate(base_stat, bonus_pool)` 获取实时属性。
* **施工方向**: 修改 Calculator，使其停止接收旧乘区对象，改为接收 `BonusPool` 或直接接收计算后的数值结果。

### 2. 角色类接口冲突 (Character Class)
**影响等级**: 🔴 **Critical** (波及所有角色模块)
* **冲突描述**:
    * **旧逻辑**: Character 直接维护 `self.buff_list` 和基础面板数值。
    * **新逻辑**: Character 持有 `BuffManager` 实例，属性变为 `DynamicAttribute` 对象。
* **施工方向**: 重写 Character `__init__`，接入 `BuffManager`。为保持兼容性，初期可保留 `buff_list` 属性但将其重定向到 `BuffManager` 的查询接口。

### 3. APL 判定系统冲突 (Simulator / APL)
**影响等级**: 🟠 **High** (导致自动战斗逻辑失效)
* **冲突描述**:
    * **旧逻辑**: APL 工具 (`FindBuff.py`) 遍历旧列表查找 Buff ID。
    * **新逻辑**: Buff 状态由 `BuffManager` 统一管理。
* **施工方向**: 重写 `FindBuff.py` 等判定工具，内部调用 `char.buff_manager.has_buff(buff_id)`。

### 4. 事件触发逻辑冲突 (BuffXLogic vs EventRouter)
**影响等级**: 🟠 **High** (逻辑双重触发或遗漏)
* **冲突描述**:
    * **旧逻辑**: 模拟器主循环显式调用 `BuffXLogic/*.py` 中的特定函数。
    * **新逻辑**: 逻辑封装在 `TriggerEffect` 中，通过 `EventRouter` 自动触发。
* **施工方向**: 清理 `Simulator` 主循环，移除所有硬编码的 Buff 调用钩子，完全转为 `event_router.publish()`。

### 5. 导入路径失效 (Import Errors)
**影响等级**: 🟡 **Medium** (工程运行报错)
* **冲突描述**: 原 `zsim.sim_progress.Buff` 下的大量文件将被移除或重命名。
* **施工方向**: 全局搜索替换。推荐使用 IDE 的重构功能将旧引用指向 `GlobalBuffController` 和 `BuffManager`。

### 6. 数据初始化冲突 (Data Init)
**影响等级**: 🟡 **Medium** (配置读取失败)
* **冲突描述**: CSV/JSON 配置文件中的字段无法直接映射到新的 `Effect` 类结构。
* **施工方向**: 在 `GlobalBuffController` 中编写适配层工厂方法，将旧配置转换为新 `Effect` 对象，而非修改原始数据文件（除非必要）。

---

## 第二部分：复杂事件触发逻辑 (BuffXLogic) 的重构方案

针对“角色触发特定层数Buff”或“特定段数攻击触发效果”等复杂逻辑，新架构采用 **组合模式 (Composition)** 与 **事件路由 (Event Routing)** 相结合的方式，避免 `TriggerEffect` 类膨胀。

### 1. 核心设计原则：组合优于继承
不再为每个 Buff 编写独立的 Python 类（如 `AliceCinema6Trigger.py`），而是通过配置组装通用的原子组件。

* **TriggerEffect (容器)**: 仅负责持有“条件”和“行为”。
* **Condition (条件)**: 原子化的判断逻辑（如“是第5层吗？”）。
* **Action (行为)**: 原子化的执行逻辑（如“造成伤害”、“清除层数”）。

**代码对比示例**:
```python
# [新方案] 通过配置组装，无需编写新类
alice_c6_effect = TriggerEffect(
    # 监听目标：自身Buff层数变更事件
    trigger_event_type=EventType.BUFF_STACK_CHANGE,
    
    # 条件列表 (AND逻辑)
    conditions=[
        TargetIsSelf(),         # 是自己
        StackCountAtLeast(5)    # 层数 >= 5
    ],
    
    # 行为列表 (顺序执行)
    action=CompositeAction([
        DealDamage(type=DmgType.ETHER, value=500), # 爆炸伤害
        ClearBuffStacks()                          # 清空层数
    ])
)
```
### 2.解决“特定段数触发”：事件画像与路由树

问题: 每次攻击都遍历所有 Buff 检查“是不是第3段”效率太低。 方案: 利用 EventRouter 的树状结构在分发阶段直接过滤。

1. EventProfile: 攻击事件生成画像 ['Attack', 'Alice', 'Skill_1', 'Segment_3']。
2. 注册: Buff 仅将监听器注册在路由树的 Segment_3 节点下。
3. 执行: 当 Segment_1 事件发生时，Router 根本不会访问该 Buff，性能零消耗。

### 3. 解决“叠层触发”：状态驱动事件
**问题**: 如何监控 Buff 层数变化？
1.  `BuffManager` 在层数改变时发布内部事件 `BuffStackChangeEvent(buff_id, new_stack)`。
2.  Buff 自身订阅该事件。
3.  通过 `Condition` 判断 `new_stack >= threshold` 来触发后续效果。

---

# ZSim Buff 系统重构：基于分支的“大爆炸”施工指南

鉴于您决定在独立 Git 分支上进行开发且不保留旧接口，本指南采用 **“底层重建 -> 逻辑迁移 -> 顶层切换”** 的策略。这种方式能最大程度保证架构的纯洁性。

---

## 阶段 1：基础设施构建 (Infrastructure Foundation)
**目标**：在不触碰现有业务代码的情况下，构建出完美的新系统内核。此时新旧代码共存但互不干扰。

1.  **构建数据模型 (`sim_progress/Buff/buff_model.py`)**:
    * 编写纯粹的 `Buff` 类。
    * 包含属性：`layer` (层数), `duration` (持续时间), `stack_id` (堆叠ID)。
    * 包含方法：`start()`, `end()`, `refresh()` 等状态变更方法。
2.  **构建核心控制器 (`sim_progress/Buff/global_buff_controller.py`)**:
    * 实现全局 Buff 注册表。
    * 实现工厂方法：读取 `zsim/data/` 下的配置并生成新的 `Buff` 对象实例。
3.  **构建效果体系 (`sim_progress/Buff/effects/`)**:
    * 实现 `EffectBase` 接口。
    * 实现 `BonusEffect` (处理数值加成)。
    * 实现 `TriggerEffect` (处理机制逻辑，作为 Condition + Action 的容器)。
4.  **构建事件路由 (`sim_progress/zsim_event_system/event_router.py`)**:
    * 实现核心的分发逻辑 (Publish/Subscribe)。
    * 实现事件画像 (`EventProfile`) 生成逻辑。

## 阶段 2：逻辑与数据迁移 (Migration)
**目标**：将散落在各处的旧逻辑“搬家”到新框架中。这是工作量最大的一步。

1.  **迁移数值 Buff**:
    * 检查 `BuffLoad.py` 或 CSV 配置。
    * 将所有简单的“攻击力+10%”类 Buff 配置映射到 `GlobalBuffController` 的加载逻辑中。
2.  **迁移机制 Buff (原 `BuffXLogic`)**:
    * 逐个分析 `zsim/sim_progress/Buff/BuffXLogic/*.py` 下的文件（如 `AliceCinema6Trigger.py`）。
    * **不要直接复制文件！** 将逻辑转化为 `TriggerEffect` 的**配置组合**。
    * *示例*：将“满层爆炸”逻辑转化为 `TriggerEffect(condition=StackFull, action=Explode)` 的初始化参数。

## 阶段 3：切换与熔断 (The Switch & Integration)
**目标**：切断旧系统，接入新系统。此步骤将导致项目暂时无法运行（大量报错）。

1.  **改造宿主 (`Character`)**:
    * **删除**：`self.buff_list` (旧列表) 及所有直接操作它的代码。
    * **新增**：`self.buff_manager = BuffManager(self)`。
2.  **改造计算器 (`Calculator`)**:
    * **修改 API**: 将 `calc_damage(..., multiplier_data)` 改为 `calc_damage(..., bonus_pool)`。
    * **重写逻辑**: 让计算器从 `bonus_pool` 中直接查询属性，不再计算旧的乘区对象。
3.  **改造模拟器 (`Simulator`)**:
    * **清理主循环**: 删除所有对 `BuffXLogic` 的显式调用。
    * **接入事件**: 在攻击发生处，插入 `event_router.publish(AttackEvent(...))`。

## 阶段 4：修复编译错误 (Fixing the Breakage)
**目标**：解决因接口删除导致的引用错误，让项目重新跑起来。

1.  利用 IDE (PyCharm/VSCode) 的错误检查，全局搜索旧 Buff 模块引用。
2.  逐个修复 APL 模块、UI 模块中对旧 Buff 系统的调用，将其重定向到 `GlobalBuffController` 或 `BuffManager`。
3.  **清理遗物**：确认系统运行正常后，直接删除 `Buff0Manager` 和 `BuffXLogic` 文件夹。

---

## 特别提示

1.  **彻底删除**：阶段 3 完成后，直接删除 `Buff0ManagerClass.py` 等旧文件，不要保留注释代码，保持新分支代码库的整洁。
2.  **单元测试先行**：在阶段 1 完成后，务必先为 `BuffManager` 和 `EventRouter` 编写单元测试，确保核心逻辑无误后再集成到 Character 中。
3.  **Calculator 过渡**：新的 `BonusPool` 结构与旧 `MultiplierData` 差异巨大。建议先写一个适配函数，将 `BonusPool` 数据临时转换为 Calculator 能接受的格式，跑通后再重构 Calculator 内部。

---

## 目标目录结构 (Target Directory Structure)

新分支下的 `zsim/sim_progress/Buff/` 目录应只包含以下内容：

```text
zsim/sim_progress/Buff/
├── buff_model.py           (新实体)
├── buff_manager.py         (新角色接口)
├── global_buff_controller.py (新仓库)
├── effects/
│   ├── base_effect.py
│   ├── bonus_effect.py
│   └── trigger_effect.py
└── (原来的 BuffXLogic, Buff0Manager 文件夹均被移除)

## 附录：Dot (异常状态) 系统重构规划

Dot (Damage over Time) 系统目前由 `BaseDot` 类及其子类（如 `Shock`, `Ignite`）管理。在旧架构中，它独立于 Buff 系统，拥有自己的一套 `start`, `end`, `ready_judge` 生命周期。
在新架构中，**异常状态（Anomaly）本质上就是一种 Buff**，其“周期性造成伤害”的特性可以通过新系统的 `TriggerEffect`（触发器效果）完美实现。

因此，本次重构将 **废弃独立的 Dot 模块**，将其完全通过 `Buff` + `EventRouter` 的形式重写。

### 1. 核心映射关系 (Mapping)

| 旧 Dot 概念 | 新系统对应实现 | 说明 |
| :--- | :--- | :--- |
| **`BaseDot` 类** | **`Buff` 模型** | Dot 不再有独立的类，而是作为 ID 特定的通用 `Buff` 实例存在（如 `Buff(id=9001, name="感电")`）。 |
| **`ft.update_cd`** (触发间隔) | **`PeriodicTrigger`** | 在 `TriggerEffect` 中增加基于时间的触发条件，或在 `Schedule` 系统中注册周期性事件。 |
| **`ft.max_duration`** | **`Buff.duration`** | 通用的 Buff 持续时间属性。 |
| **`ft.effect_rules`** (效果逻辑) | **`Effect` 组合** | 使用 `TriggerEffect` 组合 `Condition` (时间到了) 和 `Action` (造成异常伤害)。 |
| **`dy`** (动态数据) | **`Buff.dy`** | 通用的 Buff 动态数据容器。 |
| **`AnomalyBar` 交互** | **`EventHandler`** | 异常积蓄槽满时，不再 `new Shock()`，而是 `buff_manager.add_buff("Shock")`。 |

### 2. 详细施工步骤

#### 阶段 A: 基础设施扩展 (Infrastructure)

1.  **扩展 `TriggerEffect` 支持周期性事件**:
    * 在 `zsim/sim_progress/Buff/effects/trigger_effect.py` 中，设计一种机制来处理“每隔 X 秒触发一次”的逻辑。
    * **方案一 (事件驱动 - 推荐)**: `BuffManager` 在添加 Buff 时，若识别到其含有周期性 Tag，则向全局时间调度器 (`Schedule`) 注册一个循环事件 `PeriodicBuffTick(buff_id)`。
    * **方案二 (逻辑树)**: `EventRouter` 监听通用的 `TIME_TICK` 事件。*(考虑到性能，推荐方案一，避免每 Tick 遍历)*。

2.  **实现 `AnomalyDamageAction`**:
    * 在 `zsim/sim_progress/Buff/effects/actions.py` (需新建) 中实现 `DealAnomalyDamage` 行为。
    * 该行为需要能够读取快照属性（因为部分异常伤害可能基于触发时的面板快照，也可能基于实时面板，需根据游戏机制确定）。

#### 阶段 B: 数据迁移 (Migration)

1.  **配置化转化**:
    * 分析 `Shock.py`, `Ignite.py` 等文件中的 `DotFeature`。
    * 将其转化为 `GlobalBuffController` 的配置项（JSON 或工厂配置）。
    * *示例配置*:
        ```python
        # Shock (感电) 配置示例
        BuffConfig(
            id=9001,
            name="感电",
            duration=10.0,
            max_layers=1,
            effects=[
                # 效果1: 周期性伤害
                TriggerEffect(
                    trigger=PeriodicTimer(interval=1.0),
                    action=DealAnomalyDamage(type=DmgType.ELECTRIC, multiplier=1.25)
                ),
                # 效果2: 间断 (紊乱前置) - 这是一个被动特性，可能通过 Tag 实现
                TagEffect(tags=["Anomaly", "Electric", "Shock"])
            ]
        )
        ```

#### 阶段 C: 逻辑接入 (Integration)

1.  **改造 `AnomalyBar` (`zsim/sim_progress/anomaly_bar/`)**:
    * 修改 `Accumulate()` 或 `Trigger()` 逻辑。
    * **删除** 对 `BaseDot` 的引用。
    * **新增** 调用 `target_char.buff_manager.add_buff(anomaly_buff_id)`。

2.  **处理紊乱 (Disorder)**:
    * 紊乱本质上是：当添加一个新的异常 Buff 时，检测是否存在不同属性的异常 Buff。
    * 这完全可以通过 `EventRouter` 实现：
        * **监听**: `BUFF_ADDED` 事件。
        * **条件**: `AddedBuff.has_tag("Anomaly")` AND `Target.has_buff_with_tag("Anomaly")` AND `Types_Are_Different`.
        * **行为**: `TriggerDisorder()` (结算剩余伤害，清除原 Buff)。